# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,27.86,90,100,2.37,GS,1697142914
1,1,port douglas,-16.4833,145.4667,71.01,75,100,6.24,AU,1697142915
2,2,puerto baquerizo moreno,-0.9000,-89.6000,81.59,69,40,11.50,EC,1697142748
3,3,port alfred,-33.5906,26.8910,58.17,89,18,1.99,ZA,1697142917
4,4,iqaluit,63.7506,-68.5145,31.73,80,40,2.30,CA,1697142917


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoDark",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
#ideal_weather_df=city_data_df.loc[(city_data_df["Humidity"]<=70) & (city_data_df["Cloudiness"]<=80) & (city_data_df["Max Temp"]<=70)]
ideal_weather_df=city_data_df.loc[(city_data_df["Humidity"]<=70) & (city_data_df["Cloudiness"]<=80) & (city_data_df["Max Temp"]<=70) & (city_data_df["Country"]=="US")]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,albany,42.6001,-73.9662,63.59,60,52,8.12,US,1697142664
136,136,jamestown,42.0970,-79.2353,60.39,62,75,10.36,US,1697142854
243,243,broome,42.2506,-75.8330,64.78,66,75,5.99,US,1697142965
262,262,fortuna,40.5982,-124.1573,68.94,67,0,1.99,US,1697142975
267,267,palmer,42.1584,-72.3287,69.35,43,75,6.91,US,1697142732
300,300,roma,43.2128,-75.4557,60.96,62,40,10.36,US,1697142984
339,339,price,39.5994,-110.8107,54.66,35,75,12.66,US,1697142992
443,443,international falls,48.6011,-93.4110,53.15,64,75,6.91,US,1697143016
528,528,huron,41.1501,-82.5499,62.67,68,73,10.09,US,1697143035


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_weather_df.copy(deep=True)

hotel_df.reset_index
hotel_df["Hotel Name"]=""
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
45,45,albany,42.6001,-73.9662,63.59,60,52,8.12,US,1697142664,
136,136,jamestown,42.0970,-79.2353,60.39,62,75,10.36,US,1697142854,
243,243,broome,42.2506,-75.8330,64.78,66,75,5.99,US,1697142965,
262,262,fortuna,40.5982,-124.1573,68.94,67,0,1.99,US,1697142975,
267,267,palmer,42.1584,-72.3287,69.35,43,75,6.91,US,1697142732,
300,300,roma,43.2128,-75.4557,60.96,62,40,10.36,US,1697142984,
339,339,price,39.5994,-110.8107,54.66,35,75,12.66,US,1697142992,
443,443,international falls,48.6011,-93.4110,53.15,64,75,6.91,US,1697143016,
528,528,huron,41.1501,-82.5499,62.67,68,73,10.09,US,1697143035,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
#categories=accomodation.hotel

radius = 10000
category="accommodation.hotel"
latitude="Lat"
longitude="Lng"
apiKey=geoapify_key


params = {
    "categories":category,
    "apiKey":geoapify_key,
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude=row["Lng"]
    latitude=row["Lat"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
broome - nearest hotel: No hotel found
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
palmer - nearest hotel: No hotel found
roma - nearest hotel: Wingate by Wyndham Rome
price - nearest hotel: Quality Inn Price Gateway To Moab National Parks
international falls - nearest hotel: Copper River Inn & Conference Centre
huron - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
45,45,albany,42.6001,-73.9662,63.59,60,52,8.12,US,1697142664,No hotel found
136,136,jamestown,42.0970,-79.2353,60.39,62,75,10.36,US,1697142854,DoubleTree Jamestown
243,243,broome,42.2506,-75.8330,64.78,66,75,5.99,US,1697142965,No hotel found
262,262,fortuna,40.5982,-124.1573,68.94,67,0,1.99,US,1697142975,Comfort Inn & Suites Redwood Country
267,267,palmer,42.1584,-72.3287,69.35,43,75,6.91,US,1697142732,No hotel found
300,300,roma,43.2128,-75.4557,60.96,62,40,10.36,US,1697142984,Wingate by Wyndham Rome
339,339,price,39.5994,-110.8107,54.66,35,75,12.66,US,1697142992,Quality Inn Price Gateway To Moab National Parks
443,443,international falls,48.6011,-93.4110,53.15,64,75,6.91,US,1697143016,Copper River Inn & Conference Centre
528,528,huron,41.1501,-82.5499,62.67,68,73,10.09,US,1697143035,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
#poverty_rate_map = census_data_complete.hvplot.points(
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoDark",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)